## Create CNN Model and Optimize it using Keras Tuner

In [12]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
from keras.layers import Conv2D

In [13]:
print(tf.__version__)

2.12.0


In [14]:
fashion_mnist=keras.datasets.fashion_mnist

In [15]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [16]:
test_labels

array([9, 2, 1, ..., 8, 1, 5], dtype=uint8)

In [17]:
#scale down images so our model trains quickly
train_images=train_images/255.0
test_images=test_images/255.0

In [18]:
train_images[0].shape

(28, 28)

In [23]:
train_images.shape

(60000, 28, 28)

In [20]:
print(len(train_images))

60000


In [21]:
test_images.shape

(10000, 28, 28)

In [24]:
#Reshape the images for cnn
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [25]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']) 

  return model

In [28]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [29]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=2,directory='output',project_name="Mnist Fashion")

In [30]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 2 Complete [00h 05m 29s]
val_accuracy: 0.8568333387374878

Best val_accuracy So Far: 0.8611666560173035
Total elapsed time: 00h 13m 01s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
112               |48                |conv_1_filter
5                 |5                 |conv_1_kernel
64                |48                |conv_2_filter
3                 |5                 |conv_2_kernel
96                |48                |dense_1_units
0.01              |0.01              |learning_rate

Epoch 1/3


KeyboardInterrupt: 

In [31]:
model=tuner_search.get_best_models(num_models=1)[0]

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 48)        1248      
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 20, 48)        57648     
                                                                 
 flatten (Flatten)           (None, 19200)             0         
                                                                 
 dense (Dense)               (None, 48)                921648    
                                                                 
 dense_1 (Dense)             (None, 10)                490       
                                                                 
Total params: 981,034
Trainable params: 981,034
Non-trainable params: 0
_________________________________________________________________


In [33]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 138s 81ms/step - loss: 0.3644 - accuracy: 0.8678 - val_loss: 0.5176 - val_accuracy: 0.8327
Epoch 5/10
1688/1688 [==============================] - 136s 81ms/step - loss: 0.3454 - accuracy: 0.8729 - val_loss: 0.3924 - val_accuracy: 0.8640
Epoch 6/10
1688/1688 [==============================] - 142s 84ms/step - loss: 0.3309 - accuracy: 0.8782 - val_loss: 0.3764 - val_accuracy: 0.8650
Epoch 7/10
1688/1688 [==============================] - 151s 89ms/step - loss: 0.3221 - accuracy: 0.8814 - val_loss: 0.5102 - val_accuracy: 0.8387
Epoch 8/10
1688/1688 [==============================] - 155s 92ms/step - loss: 0.3081 - accuracy: 0.8874 - val_loss: 0.4632 - val_accuracy: 0.8463
Epoch 9/10
1688/1688 [==============================] - 156s 92ms/step - loss: 0.2900 - accuracy: 0.8929 - val_loss: 0.4297 - val_accuracy: 0.8660
Epoch 10/10
1688/1688 [==============================] - 328s 194ms/step - loss: 0.2908 - accuracy: 0.8933 - val_loss: